## Data Preprocessing

In [55]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [56]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:,13].values

In [57]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:,1] = labelencoder_X_1.fit_transform(X[:,1])
labelencoder_X_2 = LabelEncoder()
X[:,2] = labelencoder_X_2.fit_transform(X[:,2])

In [58]:
onehotencoder = OneHotEncoder(categorical_features=[1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:,1:] #Avoid Dummy Variable Trap

In [59]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [60]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## ANN

In [39]:
import keras
from keras.models import Sequential
from keras.layers import Dense

### Creating the ANN

In [40]:
classifier = Sequential()

In [41]:
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))

In [42]:
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))

In [43]:
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

### Compiling the ANN

In [44]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Fitting the ANN

In [45]:
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 1s 137us/step - loss: 0.5001 - acc: 0.7952
Epoch 2/100
8000/8000 [==============================] - 1s 95us/step - loss: 0.4303 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 123us/step - loss: 0.4241 - acc: 0.7989
Epoch 4/100
8000/8000 [==============================] - 1s 112us/step - loss: 0.4172 - acc: 0.8257
Epoch 5/100
8000/8000 [==============================] - 1s 153us/step - loss: 0.4121 - acc: 0.8311
Epoch 6/100
8000/8000 [==============================] - 1s 130us/step - loss: 0.4081 - acc: 0.8320
Epoch 7/100
8000/8000 [==============================] - 1s 120us/step - loss: 0.4055 - acc: 0.8346
Epoch 8/100
8000/8000 [==============================] - 1s 110us/step - loss: 0.4038 - acc: 0.8349
Epoch 9/100
8000/8000 [==============================] - 1s 138us/step - loss: 0.4026 - acc: 0.8352
Epoch 10/100
8000/8000 [==============================] - 1s 145us/step - loss: 0.4012 - acc: 0.8362


## Predictions

In [47]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [49]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1510,   85],
       [ 195,  210]])

In [52]:
accuracy = (cm[0][0]+cm[1][1])/(cm[0][0]+cm[0][1]+cm[1][0]+cm[1][1])
accuracy

0.86

### Predict for one new example
- Geography: France
- Credit Score: 600
- Tenure: 3
- Gender: Male
- Age: 40
- Balance: 60000
- Number of Products: 2
- Has Credit Card: Yes
- Is Active Member: Yes
- Estimated Salary: 50000

In [54]:
new_info = sc.transform(np.array([[0.0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]))
new_pred = classifier.predict(new_info)
new_pred = (new_pred > 0.5)
new_pred

array([[False]])

## Evaluating the ANN

In [63]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 100)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = 3)

In [ ]:
mean = accuracies.mean()
variance = accuracies.std()

## Improving the ANN(Dropout)

In [68]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

In [71]:
classifier = Sequential()

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
classifier.add(Dropout(rate = 0.1))

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dropout(rate = 0.1))

classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 1s 164us/step - loss: 0.4883 - acc: 0.7956
Epoch 2/100
8000/8000 [==============================] - 1s 112us/step - loss: 0.4383 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 113us/step - loss: 0.4335 - acc: 0.7960
Epoch 4/100
8000/8000 [==============================] - 1s 116us/step - loss: 0.4272 - acc: 0.7960
Epoch 5/100
8000/8000 [==============================] - 1s 125us/step - loss: 0.4280 - acc: 0.8104
Epoch 6/100
8000/8000 [==============================] - 1s 114us/step - loss: 0.4271 - acc: 0.8245
Epoch 7/100
8000/8000 [==============================] - 1s 116us/step - loss: 0.4258 - acc: 0.8266
Epoch 8/100
8000/8000 [==============================] - 1s 113us/step - loss: 0.4277 - acc: 0.8266
Epoch 9/100
8000/8000 [==============================] - 1s 114us/step - loss: 0.4255 - acc: 0.8269
Epoch 10/100
8000/8000 [==============================] - 1s 115us/step - loss: 0.4260 - acc: 0.8286

## Improving the ANN

In [76]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [79]:
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier)

In [80]:
parameters = {
    'batch_size': [25, 32],
    'epochs': [100, 500],
    'optimizer': ['adam', 'rmsprop']
}

In [81]:
grid_search = GridSearchCV(estimator = classifier, param_grid = parameters, scoring = 'accuracy', cv = 10)

In [82]:
grid_search = grid_search.fit(X_train, y_train)

Epoch 1/100
7200/7200 [==============================] - 1s 92us/step - loss: 0.6025 - acc: 0.7953
Epoch 2/100
7200/7200 [==============================] - 0s 46us/step - loss: 0.4407 - acc: 0.7971
Epoch 3/100
7200/7200 [==============================] - 0s 46us/step - loss: 0.4189 - acc: 0.8133
Epoch 4/100
7200/7200 [==============================] - 0s 41us/step - loss: 0.4089 - acc: 0.8243
Epoch 5/100
7200/7200 [==============================] - 0s 45us/step - loss: 0.4008 - acc: 0.8282
Epoch 6/100
7200/7200 [==============================] - 0s 46us/step - loss: 0.3946 - acc: 0.8292
Epoch 7/100
7200/7200 [==============================] - 0s 41us/step - loss: 0.3899 - acc: 0.8289
Epoch 8/100
7200/7200 [==============================] - 0s 42us/step - loss: 0.3858 - acc: 0.8307
Epoch 9/100
7200/7200 [==============================] - 0s 47us/step - loss: 0.3826 - acc: 0.8324
Epoch 10/100
7200/7200 [==============================] - 0s 42us/step - loss: 0.3788 - acc: 0.8333
Epoch 11/

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



7200/7200 [==============================] - 1s 112us/step - loss: 0.4070 - acc: 0.8353
Epoch 29/500
7200/7200 [==============================] - 1s 99us/step - loss: 0.4066 - acc: 0.8339
Epoch 30/500
7200/7200 [==============================] - 1s 105us/step - loss: 0.4066 - acc: 0.8361
Epoch 31/500
7200/7200 [==============================] - 1s 100us/step - loss: 0.4062 - acc: 0.8367
Epoch 32/500
7200/7200 [==============================] - 1s 91us/step - loss: 0.4061 - acc: 0.8354
Epoch 33/500
7200/7200 [==============================] - 1s 90us/step - loss: 0.4056 - acc: 0.8351
Epoch 34/500
7200/7200 [==============================] - 1s 98us/step - loss: 0.4058 - acc: 0.8354
Epoch 35/500
7200/7200 [==============================] - 1s 103us/step - loss: 0.4054 - acc: 0.8362
Epoch 36/500
7200/7200 [==============================] - 1s 97us/step - loss: 0.4054 - acc: 0.8354
Epoch 37/500
7200/7200 [==============================] - 1s 102us/step - loss: 0.4049 - acc: 0.8350
Epoch 38

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



7200/7200 [==============================] - 1s 94us/step - loss: 0.3416 - acc: 0.8585
Epoch 59/500
7200/7200 [==============================] - 1s 93us/step - loss: 0.3411 - acc: 0.8594
Epoch 60/500
7200/7200 [==============================] - 1s 90us/step - loss: 0.3410 - acc: 0.8596
Epoch 61/500
7200/7200 [==============================] - 1s 93us/step - loss: 0.3408 - acc: 0.8593
Epoch 62/500
7200/7200 [==============================] - 1s 91us/step - loss: 0.3402 - acc: 0.8594
Epoch 63/500
7200/7200 [==============================] - 1s 91us/step - loss: 0.3405 - acc: 0.8585
Epoch 64/500
7200/7200 [==============================] - 1s 97us/step - loss: 0.3402 - acc: 0.8587
Epoch 65/500
7200/7200 [==============================] - 1s 96us/step - loss: 0.3395 - acc: 0.8603
Epoch 66/500
7200/7200 [==============================] - 1s 89us/step - loss: 0.3392 - acc: 0.8604
Epoch 67/500
7200/7200 [==============================] - 1s 96us/step - loss: 0.3393 - acc: 0.8587
Epoch 68/500


...

7200/7200 [==============================] - 1s 114us/step - loss: 0.3991 - acc: 0.8353
Epoch 274/500
7200/7200 [==============================] - 1s 97us/step - loss: 0.3993 - acc: 0.8343
Epoch 275/500
7200/7200 [==============================] - 1s 103us/step - loss: 0.3991 - acc: 0.8337
Epoch 276/500
7200/7200 [==============================] - 1s 114us/step - loss: 0.3993 - acc: 0.8343
Epoch 277/500
7200/7200 [==============================] - 1s 104us/step - loss: 0.3992 - acc: 0.8353
Epoch 278/500
7200/7200 [==============================] - 1s 106us/step - loss: 0.3990 - acc: 0.8336
Epoch 279/500
7200/7200 [==============================] - 1s 103us/step - loss: 0.3990 - acc: 0.8358
Epoch 280/500
7200/7200 [==============================] - 1s 104us/step - loss: 0.3994 - acc: 0.8336
Epoch 281/500
7200/7200 [==============================] - 1s 108us/step - loss: 0.3992 - acc: 0.8344
Epoch 282/500
7200/7200 [==============================] - 1s 99us/step - loss: 0.3993 - acc: 0.8

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



7200/7200 [==============================] - 1s 127us/step - loss: 0.3949 - acc: 0.8364
Epoch 145/500
7200/7200 [==============================] - 1s 111us/step - loss: 0.3948 - acc: 0.8365
Epoch 146/500
7200/7200 [==============================] - 1s 109us/step - loss: 0.3947 - acc: 0.8371
Epoch 147/500
7200/7200 [==============================] - 1s 116us/step - loss: 0.3945 - acc: 0.8368
Epoch 148/500
7200/7200 [==============================] - 1s 116us/step - loss: 0.3949 - acc: 0.8376
Epoch 149/500
7200/7200 [==============================] - 1s 115us/step - loss: 0.3946 - acc: 0.8360
Epoch 150/500
7200/7200 [==============================] - 1s 108us/step - loss: 0.3947 - acc: 0.8375
Epoch 151/500
7200/7200 [==============================] - 1s 111us/step - loss: 0.3951 - acc: 0.8364
Epoch 152/500
7200/7200 [==============================] - 1s 120us/step - loss: 0.3946 - acc: 0.8369
Epoch 153/500
7200/7200 [==============================] - 1s 108us/step - loss: 0.3949 - acc: 0

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



7200/7200 [==============================] - 1s 119us/step - loss: 0.3311 - acc: 0.8664
Epoch 437/500
7200/7200 [==============================] - 1s 123us/step - loss: 0.3317 - acc: 0.8647
Epoch 438/500
7200/7200 [==============================] - 1s 121us/step - loss: 0.3315 - acc: 0.8649
Epoch 439/500
7200/7200 [==============================] - 1s 114us/step - loss: 0.3315 - acc: 0.8653
Epoch 440/500
7200/7200 [==============================] - 1s 119us/step - loss: 0.3318 - acc: 0.8647
Epoch 441/500
7200/7200 [==============================] - 1s 114us/step - loss: 0.3316 - acc: 0.8660
Epoch 442/500
7200/7200 [==============================] - 1s 120us/step - loss: 0.3317 - acc: 0.8653
Epoch 443/500
7200/7200 [==============================] - 1s 118us/step - loss: 0.3319 - acc: 0.8654
Epoch 444/500
7200/7200 [==============================] - 1s 115us/step - loss: 0.3314 - acc: 0.8653
Epoch 445/500
7200/7200 [==============================] - 1s 118us/step - loss: 0.3317 - acc: 0

In [83]:
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

In [85]:
best_parameters

{'batch_size': 25, 'epochs': 500, 'optimizer': 'rmsprop'}

In [86]:
best_accuracy

0.8545